In [ ]:
df = read.csv('hmeq.csv', header = TRUE)

In [ ]:
## 1-2
df=df[complete.cases(df), ]
sum(is.na(df))
df$BAD = as.factor(df$BAD)
names(df)[1] = "target"
summary(df)

In [ ]:
## 3
library(caret)
splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]

In [ ]:
## 4
library(ranger)
model = ranger(target ~., data = train)
pred = predict(model,data = test)$predictions
cm = confusionMatrix(pred, test$target, positive = "1")
cm

#### Accuracy = 0.9459   Balanced Accuracy = 0.69513

In [ ]:
## 5
table(df$target)
### ratio of default vs non-default clients. Way more 0's then 1's.

In [ ]:
## 6
train0 = train[train$target == '0',]
train1 = train[train$target == '1',]
n0 = nrow(train0)
n1 = nrow(train1)
train00 = train0[sample(1:n0, n1),]
train_under = rbind(train00, train1)
model_under = ranger(target~., data = train_under)
pred  = predict(model_under, data = test)$predictions
cm_under=confusionMatrix(pred, test$target, positive="1")
print(cm_under)

#### Accuracy = 0.8082   Balance Accuracy = 0.78678

In [ ]:
## 7
train11 = train1[sample(1:n1, n0, replace = TRUE),]
train_over = rbind(train11, train0)
model_over = ranger(target~., data = train_over)
pred_over  = predict(model_over, data = test)$predictions
cm_over=confusionMatrix(pred_over, test$target, positive="1")
print(cm_over)

#### Accuracy = 0.944   Balanced Accuracy = 0.68918

In [ ]:
## 8
Quick_bal = function(x, undersampling = TRUE) {
  library(caret)
  library(ranger)
  if(undersampling == TRUE){
    splitIndex <- createDataPartition(x$target, p = .70, list = FALSE, times = 1)
    train <- x[splitIndex,]
    test <- x[-splitIndex,]
    train0 = train[train$target == '0',]
    train1 = train[train$target == '1',]
    n0 = nrow(train0)
    n1 = nrow(train1)
    train00 = train0[sample(1:n0, n1),]
    train_under = rbind(train00, train1)
    return(train_under)
  }
  if(undersampling == FALSE ){
   splitIndex <- createDataPartition(x$target, p = .70, list = FALSE, times = 1)
    train <- x[splitIndex,]
    test <- x[-splitIndex,]
    train0 = train[train$target == '0',]
    train1 = train[train$target == '1',]
    n0 = nrow(train0)
    n1 = nrow(train1)
    train11 = train1[sample(1:n1, n0, replace = TRUE),]
    train_over = rbind(train11, train0)
    return(train_over)
  }
}

In [ ]:
## 9
Quick_Model_Bal = function(x){
  library(caret)
  library(ranger)
  
    splitIndex <- createDataPartition(x$target, p = .70, list = FALSE, times = 1)
    train <- x[splitIndex,]
    test <- x[-splitIndex,]
    train0 = train[train$target == '0',]
    train1 = train[train$target == '1',]
    n0 = nrow(train0)
    n1 = nrow(train1)
    train00 = train0[sample(1:n0, n1),]
    train_under = rbind(train00, train1)
    model_under = ranger(target~., data = train_under)
    pred  = predict(model_under, data = test)$predictions
    cm_under=confusionMatrix(pred, test$target, positive="1")
    print("Undersampling Model")
    print(cm_under)
    
    train11 = train1[sample(1:n1, n0, replace = TRUE),]
    train_over = rbind(train11, train0)
    model_over = ranger(target~., data = train_over)
    pred_over  = predict(model_over, data = test)$predictions
    cm_over=confusionMatrix(pred_over, test$target, positive="1")
    print("Oversampling Model")
    print(cm_over)
}

In [ ]:
Quick_Model_Bal(df)